In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
con = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/content_paper_score.csv', encoding='cp949')
con.columns = ['subsr', 'program', 'vod_id', 'caper_score', 'main_cat', 'sub_cat', 'category']
con

,subsr,program,vod_id,caper_score,main_cat,sub_cat,category
0,59879000,나쁜엄마,3403,0.187500,TV드라마,기타,TV프로그램
1,59879000,소방서 옆 경찰서,2092,0.111110,TV드라마,기타,TV프로그램
2,59879000,낭만닥터 김사부2,1168,0.106201,TV드라마,기타,TV프로그램
3,59879000,소방서 옆 경찰서,2092,0.222220,TV드라마,기타,TV프로그램
4,59879000,신성한 이혼,1190,0.145833,TV드라마,기타,TV프로그램
...,...,...,...,...,...,...,...
9485,67160000,콘크리트 유토피아,2630,0.125000,영화,드라마,영화
9486,67160000,스파이더맨: 어크로스 더 유니버스,2147,0.124989,영화,애니메이션,영화
9487,67160000,메가로돈 2,4018,0.125000,영화,액션/어드벤쳐,영화
9488,67161000,스파이 코드명 포춘,3824,0.999989,영화,액션/어드벤쳐,영화


In [ ]:
vod_list = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/vod_list_add10.csv', index_col=0)
vod_id = vod_list[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [ ]:
con1 = con.copy()
con1

,subsr,program,vod_id,caper_score,main_cat,sub_cat,category
0,59879000,나쁜엄마,3403,0.187500,TV드라마,기타,TV프로그램
1,59879000,소방서 옆 경찰서,2092,0.111110,TV드라마,기타,TV프로그램
2,59879000,낭만닥터 김사부2,1168,0.106201,TV드라마,기타,TV프로그램
3,59879000,소방서 옆 경찰서,2092,0.222220,TV드라마,기타,TV프로그램
4,59879000,신성한 이혼,1190,0.145833,TV드라마,기타,TV프로그램
...,...,...,...,...,...,...,...
9485,67160000,콘크리트 유토피아,2630,0.125000,영화,드라마,영화
9486,67160000,스파이더맨: 어크로스 더 유니버스,2147,0.124989,영화,애니메이션,영화
9487,67160000,메가로돈 2,4018,0.125000,영화,액션/어드벤쳐,영화
9488,67161000,스파이 코드명 포춘,3824,0.999989,영화,액션/어드벤쳐,영화


In [ ]:
con_score = con1[['subsr', 'vod_id', 'caper_score']]
con_score

,subsr,vod_id,caper_score
0,59879000,3403,0.187500
1,59879000,2092,0.111110
2,59879000,1168,0.106201
3,59879000,2092,0.222220
4,59879000,1190,0.145833
...,...,...,...
9485,67160000,2630,0.125000
9486,67160000,2147,0.124989
9487,67160000,4018,0.125000
9488,67161000,3824,0.999989


In [ ]:
! pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163758 sha256=88577469f7e788b7566d627c4d99a7784b629fdfe0d292ff08697d14e8d17dac
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 범위가 0~1 인 경우
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'caper_score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(con_score)

### KNNBaseline - user-based similarity
- cosine (0.0010309278350515464)
- **pearson_baseline (0.003608247422680412)**
- msd (0.0010309278350515464)

In [ ]:
sim_options = {'name': 'msd',
               'user_based': True  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.260881
1,59879000,3,None,0.275125
2,59879000,5,None,0.217675
3,59879000,6,None,0.211425
4,59879000,7,None,0.214976
...,...,...,...,...
2045487,67164000,4701,None,0.249216
2045488,67164000,4706,None,0.248221
2045489,67164000,4710,None,0.265235
2045490,67164000,4712,None,0.248221


In [ ]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.260881
1,59879000,3,0.275125
2,59879000,5,0.217675
3,59879000,6,0.211425
4,59879000,7,0.214976
...,...,...,...
2045487,67164000,4701,0.249216
2045488,67164000,4706,0.248221
2045489,67164000,4710,0.265235
2045490,67164000,4712,0.248221


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2327,4666,496,1970,3223,2978,1486,1025,3710,...,1712,4025,3624,3089,2329,1411,1868,1419,1423,3535
1,59882000,610,1310,2637,1394,986,719,4666,1555,1406,...,1410,1539,3292,2313,3467,4113,1278,4483,1582,2978
2,59886000,610,2327,496,3223,89,4089,37,3175,3113,...,1954,3566,2726,2571,723,80,54,917,96,83
3,59890000,1045,4502,2762,2963,2702,2880,626,1151,2360,...,1215,2598,180,218,2608,2867,2679,1329,2578,3516
4,59892000,2151,60,3112,1310,547,544,2637,3029,3291,...,4289,4301,491,1045,2686,981,1563,713,4127,1503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,67149000,1310,2327,547,3546,2082,860,703,2974,2613,...,1410,1078,4293,4156,2455,2313,496,1970,3223,846
862,67154000,2082,3425,2533,3223,1106,2290,2243,2091,1798,...,1205,4269,3559,3602,485,3689,2867,652,1120,4375
863,67160000,2151,3112,1310,4107,2327,544,2637,3546,2082,...,981,1563,4127,2620,1078,1410,1539,4293,4156,3747
864,67161000,4107,1128,3546,2613,2974,2686,1971,2799,220,...,4507,1953,3989,2157,635,4276,2229,604,616,2289


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2327, 4666, 496, 1970, 3223, 2978, 1486, 1025..."
1,59882000,"[610, 1310, 2637, 1394, 986, 719, 4666, 1555, ..."
2,59886000,"[610, 2327, 496, 3223, 89, 4089, 37, 3175, 311..."
3,59890000,"[1045, 4502, 2762, 2963, 2702, 2880, 626, 1151..."
4,59892000,"[2151, 60, 3112, 1310, 547, 544, 2637, 3029, 3..."
...,...,...
861,67149000,"[1310, 2327, 547, 3546, 2082, 860, 703, 2974, ..."
862,67154000,"[2082, 3425, 2533, 3223, 1106, 2290, 2243, 209..."
863,67160000,"[2151, 3112, 1310, 4107, 2327, 544, 2637, 3546..."
864,67161000,"[4107, 1128, 3546, 2613, 2974, 2686, 1971, 279..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2327.0, 4666.0, 496.0, 1970.0, 3223.0, 2978.0..."
1,59882000,"[610.0, 1310.0, 2637.0, 1394.0, 986.0, 719.0, ..."
2,59886000,"[610.0, 2327.0, 496.0, 3223.0, 89.0, 4089.0, 3..."
3,59890000,"[1045.0, 4502.0, 2762.0, 2963.0, 2702.0, 2880...."
4,59892000,"[2151.0, 60.0, 3112.0, 1310.0, 547.0, 544.0, 2..."
...,...,...
861,67149000,"[1310.0, 2327.0, 547.0, 3546.0, 2082.0, 860.0,..."
862,67154000,"[2082.0, 3425.0, 2533.0, 3223.0, 1106.0, 2290...."
863,67160000,"[2151.0, 3112.0, 1310.0, 4107.0, 2327.0, 544.0..."
864,67161000,"[4107.0, 1128.0, 3546.0, 2613.0, 2974.0, 2686...."


In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.0010309278350515464

### KNNBaseline - item-based similarity
- cosine (0.003608247422680412)
- **pearson_baseline (0.00927835051546392)**
- msd (0.007731958762886598)

In [ ]:
sim_options = {'name': 'msd',
               'user_based': False  # user-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline = KNNBaseline(k = 6, sim_options=sim_options, random_state = 42)
knnbaseline.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.260881
1,59879000,3,None,0.275125
2,59879000,5,None,0.217675
3,59879000,6,None,0.279330
4,59879000,7,None,0.214976
...,...,...,...,...
2045487,67164000,4701,None,0.249216
2045488,67164000,4706,None,0.248221
2045489,67164000,4710,None,0.265235
2045490,67164000,4712,None,0.248221


In [ ]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.260881
1,59879000,3,0.275125
2,59879000,5,0.217675
3,59879000,6,0.279330
4,59879000,7,0.214976
...,...,...,...
2045487,67164000,4701,0.249216
2045488,67164000,4706,0.248221
2045489,67164000,4710,0.265235
2045490,67164000,4712,0.248221


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2537,2288,2057,2867,2608,1329,240,2556,2024,...,293,1970,6,2343,575,82,3,2151,60,1193
1,59882000,2537,2288,2057,1167,2608,1329,1278,1202,2380,...,320,1172,3645,3083,493,2250,3694,1234,1485,2042
2,59886000,2867,2630,2057,610,3175,2537,2608,2588,168,...,1267,7,613,2846,818,1629,618,3,3850,2327
3,59890000,2537,2556,2630,2057,2288,2633,4114,759,1045,...,4018,626,1151,4138,2058,3360,2157,2818,2360,1666
4,59892000,2537,2024,2288,1329,547,1122,3285,2057,0,...,2608,1046,821,2678,1353,750,1189,2794,4617,4411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,67149000,2537,2288,2057,2608,2867,1329,2447,4127,2630,...,4195,3659,847,3697,2151,504,60,1798,3414,2290
862,67154000,2091,1597,1885,1848,1802,1798,1790,259,1775,...,1428,1391,1895,2033,2537,2288,168,169,2445,2433
863,67160000,2537,2288,1329,2057,2686,87,2447,3588,4435,...,3298,3388,1974,2517,1282,1185,3285,986,3479,727
864,67161000,2535,2633,4276,220,1391,1971,2867,4192,2630,...,87,4107,4103,1215,2810,1673,2677,2682,4018,2799


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2537, 2288, 2057, 2867, 2608, 1329, 240, 2556..."
1,59882000,"[2537, 2288, 2057, 1167, 2608, 1329, 1278, 120..."
2,59886000,"[2867, 2630, 2057, 610, 3175, 2537, 2608, 2588..."
3,59890000,"[2537, 2556, 2630, 2057, 2288, 2633, 4114, 759..."
4,59892000,"[2537, 2024, 2288, 1329, 547, 1122, 3285, 2057..."
...,...,...
861,67149000,"[2537, 2288, 2057, 2608, 2867, 1329, 2447, 412..."
862,67154000,"[2091, 1597, 1885, 1848, 1802, 1798, 1790, 259..."
863,67160000,"[2537, 2288, 1329, 2057, 2686, 87, 2447, 3588,..."
864,67161000,"[2535, 2633, 4276, 220, 1391, 1971, 2867, 4192..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2537.0, 2288.0, 2057.0, 2867.0, 2608.0, 1329...."
1,59882000,"[2537.0, 2288.0, 2057.0, 1167.0, 2608.0, 1329...."
2,59886000,"[2867.0, 2630.0, 2057.0, 610.0, 3175.0, 2537.0..."
3,59890000,"[2537.0, 2556.0, 2630.0, 2057.0, 2288.0, 2633...."
4,59892000,"[2024.0, 2288.0, 1329.0, 547.0, 1122.0, 3285.0..."
...,...,...
861,67149000,"[2537.0, 2288.0, 2608.0, 2867.0, 1329.0, 2447...."
862,67154000,"[2091.0, 1597.0, 1885.0, 1848.0, 1802.0, 1798...."
863,67160000,"[2537.0, 2288.0, 1329.0, 2686.0, 87.0, 2447.0,..."
864,67161000,"[2535.0, 2633.0, 4276.0, 220.0, 1391.0, 1971.0..."


In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.007731958762886598

### BaselineOnly

In [ ]:
bsl_options = {'method' : 'als', 'n_epochs' : 60}
baseline = BaselineOnly(bsl_options=bsl_options)
baseline.fit(trainset)

Estimating biases using als...


In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(baseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.260897
1,59879000,3,0.275141
2,59879000,5,0.217674
3,59879000,6,0.240056
4,59879000,7,0.214982
...,...,...,...
2045487,67164000,4701,0.249216
2045488,67164000,4706,0.248224
2045489,67164000,4710,0.265235
2045490,67164000,4712,0.248224


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
1,59882000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
2,59886000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
3,59890000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
4,59892000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,67149000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
862,67154000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
863,67160000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565
864,67161000,2537,2288,2057,2608,2867,1329,240,2556,2024,...,333,3112,1877,1310,2327,547,4664,544,2637,565


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
1,59882000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
2,59886000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
3,59890000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
4,59892000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
...,...,...
861,67149000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
862,67154000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
863,67160000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."
864,67161000,"[2537, 2288, 2057, 2608, 2867, 1329, 240, 2556..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict, K = 10)

In [18]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [19]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.0005154639175257732

### SVD
- basic (0.0010309278350515464)
- n_epochs= 160, lr_all= 0.03, reg_all= 0.01, random_state= 42 (0.0010309278350515464)
- n_epochs= 195, lr_all= 0.03, reg_all= 0.01, random_state= 42 (0.0005154639175257732)


In [46]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(150, 250, 5), 'lr_all': [0.01, 0.05, 0.005],
              'reg_all': [0.01, 0.05, 0.01], 'random_state' : [42], 'n_factors' : [200], 'n_factors': [100, 200, 300]}

              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'caper_score']], reader)

gs.fit(train_set)
gs.best_params

KeyboardInterrupt: ignored

In [47]:
svd = SVD(n_epochs= 195, lr_all= 0.03, reg_all= 0.01, random_state= 42)
svd.fit(trainset)

In [48]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.482633
1,59879000,3,0.364669
2,59879000,5,0.276214
3,59879000,6,0.360197
4,59879000,7,0.114661
...,...,...,...
2045487,67164000,4701,0.493125
2045488,67164000,4706,0.508149
2045489,67164000,4710,0.663082
2045490,67164000,4712,0.793638


In [49]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,60,3029,2327,860,430,1555,1539,547,1167,...,1078,2817,3439,2082,2637,4293,2799,231,815,1926
1,59882000,547,860,3291,2637,2327,2620,1406,60,1025,...,1926,1503,300,1410,3425,430,265,1412,146,3717
2,59886000,60,2637,2620,3226,1539,3029,1013,3283,547,...,74,2082,1845,3178,1406,3371,1280,3439,4181,2753
3,59890000,1078,1555,3439,547,559,4293,60,3546,2620,...,1410,634,3425,1406,37,300,948,2313,2094,496
4,59892000,1406,2094,547,2637,860,265,948,60,4666,...,3717,3747,4085,815,1926,2620,3425,2686,1555,3581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,67149000,547,60,3747,2620,860,3581,2327,1926,948,...,2553,331,4293,491,1876,430,3634,2637,163,4156
862,67154000,0,1587,2051,1845,1819,1738,163,1678,1563,...,2343,2391,2469,2535,2546,2553,2560,2573,430,2637
863,67160000,2637,547,60,3029,3291,860,2082,3439,430,...,3804,1167,163,231,491,1406,4502,4085,1555,1503
864,67161000,587,1926,1113,1167,60,3965,496,491,1406,...,2327,3291,2455,220,231,235,2620,3112,2687,300


In [50]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[60, 3029, 2327, 860, 430, 1555, 1539, 547, 11..."
1,59882000,"[547, 860, 3291, 2637, 2327, 2620, 1406, 60, 1..."
2,59886000,"[60, 2637, 2620, 3226, 1539, 3029, 1013, 3283,..."
3,59890000,"[1078, 1555, 3439, 547, 559, 4293, 60, 3546, 2..."
4,59892000,"[1406, 2094, 547, 2637, 860, 265, 948, 60, 466..."
...,...,...
861,67149000,"[547, 60, 3747, 2620, 860, 3581, 2327, 1926, 9..."
862,67154000,"[0, 1587, 2051, 1845, 1819, 1738, 163, 1678, 1..."
863,67160000,"[2637, 547, 60, 3029, 3291, 860, 2082, 3439, 4..."
864,67161000,"[587, 1926, 1113, 1167, 60, 3965, 496, 491, 14..."


In [51]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[60.0, 3029.0, 2327.0, 860.0, 430.0, 1555.0, 1..."
1,59882000,"[547.0, 860.0, 3291.0, 2637.0, 2327.0, 2620.0,..."
2,59886000,"[60.0, 2637.0, 2620.0, 3226.0, 1539.0, 3029.0,..."
3,59890000,"[1078.0, 1555.0, 3439.0, 547.0, 559.0, 4293.0,..."
4,59892000,"[1406.0, 2094.0, 547.0, 2637.0, 860.0, 265.0, ..."
...,...,...
861,67149000,"[547.0, 60.0, 3747.0, 2620.0, 860.0, 3581.0, 2..."
862,67154000,"[0.0, 1587.0, 2051.0, 1845.0, 1819.0, 1738.0, ..."
863,67160000,"[2637.0, 547.0, 60.0, 3029.0, 3291.0, 860.0, 2..."
864,67161000,"[587.0, 1926.0, 1113.0, 1167.0, 60.0, 3965.0, ..."


In [54]:
merge_df = pd.merge(testdata, vod_predict_1, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
merge_df

,subsr,vod_id_actual,vod_id_rec
0,59900000,"[1278, 1885]","[60.0, 547.0, 2637.0, 860.0, 2327.0, 3291.0, 3..."
1,59930000,[2098],"[2637.0, 547.0, 3291.0, 860.0, 2620.0, 2327.0,..."
2,59933000,[296],"[547.0, 3029.0, 60.0, 3291.0, 1406.0, 1410.0, ..."
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]","[60.0, 2637.0, 547.0, 860.0, 3291.0, 2327.0, 2..."
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,...","[547.0, 60.0, 2637.0, 3029.0, 3291.0, 860.0, 2..."
...,...,...,...
215,67133000,[296],NaN
216,67164000,"[4956, 1434, 4995, 2058]","[2523.0, 3439.0, 3467.0, 3414.0, 3291.0, 3283...."
217,67170000,"[3464, 948, 3900, 675]",NaN
218,67202000,[1028],NaN


In [52]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.0005154639175257732

### SVDpp
-

In [26]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(120, 150, 5), 'lr_all': [0.01, 0.05],
              'reg_all': [0.05, 0.09], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'caper_score']], reader)

gs.fit(train_set)
gs.best_params

{'mae': {'n_epochs': 145, 'lr_all': 0.01, 'reg_all': 0.05, 'random_state': 42}}

In [30]:
svdpp = SVDpp(n_epochs= 145, lr_all= 0.01, reg_all= 0.05, random_state= 42)
svdpp.fit(trainset)

In [31]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svdpp.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.528002
1,59879000,3,0.331047
2,59879000,5,0.256752
3,59879000,6,0.269312
4,59879000,7,0.152962
...,...,...,...
2045487,67164000,4701,0.788452
2045488,67164000,4706,0.624042
2045489,67164000,4710,0.812976
2045490,67164000,4712,0.720909


In [66]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,60,3029,2327,860,430,1555,1539,547,1167,...,1078,2817,3439,2082,2637,4293,2799,231,815,1926
1,59882000,547,860,3291,2637,2327,2620,1406,60,1025,...,1926,1503,300,1410,3425,430,265,1412,146,3717
2,59886000,60,2637,2620,3226,1539,3029,1013,3283,547,...,74,2082,1845,3178,1406,3371,1280,3439,4181,2753
3,59890000,1078,1555,3439,547,559,4293,60,3546,2620,...,1410,634,3425,1406,37,300,948,2313,2094,496
4,59892000,1406,2094,547,2637,860,265,948,60,4666,...,3717,3747,4085,815,1926,2620,3425,2686,1555,3581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,67149000,547,60,3747,2620,860,3581,2327,1926,948,...,2553,331,4293,491,1876,430,3634,2637,163,4156
862,67154000,0,1587,2051,1845,1819,1738,163,1678,1563,...,2343,2391,2469,2535,2546,2553,2560,2573,430,2637
863,67160000,2637,547,60,3029,3291,860,2082,3439,430,...,3804,1167,163,231,491,1406,4502,4085,1555,1503
864,67161000,587,1926,1113,1167,60,3965,496,491,1406,...,2327,3291,2455,220,231,235,2620,3112,2687,300


In [67]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[60, 3029, 2327, 860, 430, 1555, 1539, 547, 11..."
1,59882000,"[547, 860, 3291, 2637, 2327, 2620, 1406, 60, 1..."
2,59886000,"[60, 2637, 2620, 3226, 1539, 3029, 1013, 3283,..."
3,59890000,"[1078, 1555, 3439, 547, 559, 4293, 60, 3546, 2..."
4,59892000,"[1406, 2094, 547, 2637, 860, 265, 948, 60, 466..."
...,...,...
861,67149000,"[547, 60, 3747, 2620, 860, 3581, 2327, 1926, 9..."
862,67154000,"[0, 1587, 2051, 1845, 1819, 1738, 163, 1678, 1..."
863,67160000,"[2637, 547, 60, 3029, 3291, 860, 2082, 3439, 4..."
864,67161000,"[587, 1926, 1113, 1167, 60, 3965, 496, 491, 14..."


In [68]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

svdpp_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    svdpp_predict_1 = pd.concat([svdpp_predict_1, pd.DataFrame(li).T], axis= 0)

svdpp_predict_1.index = user_id
svdpp_predict_1 = svdpp_predict_1.apply(lambda x : x.tolist(), axis = 1)
svdpp_predict_1 = svdpp_predict_1.reset_index()
svdpp_predict_1.columns = ['subsr', 'vod_id']
svdpp_predict_1

,subsr,vod_id
0,59879000,"[60.0, 3029.0, 2327.0, 860.0, 430.0, 1555.0, 1..."
1,59882000,"[547.0, 860.0, 3291.0, 2637.0, 2327.0, 2620.0,..."
2,59886000,"[60.0, 2637.0, 2620.0, 3226.0, 1539.0, 3029.0,..."
3,59890000,"[1078.0, 1555.0, 3439.0, 547.0, 559.0, 4293.0,..."
4,59892000,"[1406.0, 2094.0, 547.0, 2637.0, 860.0, 265.0, ..."
...,...,...
861,67149000,"[547.0, 60.0, 3747.0, 2620.0, 860.0, 3581.0, 2..."
862,67154000,"[0.0, 1587.0, 2051.0, 1845.0, 1819.0, 1738.0, ..."
863,67160000,"[2637.0, 547.0, 60.0, 3029.0, 3291.0, 860.0, 2..."
864,67161000,"[587.0, 1926.0, 1113.0, 1167.0, 60.0, 3965.0, ..."


In [57]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [69]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, svdpp_predict_1)

0.0005154639175257732